# Edge AI Data Exploration Notebook

This Jupyter Notebook demonstrates how to read **all supported Edge AI data formats**:

- JSONL / JSON
- Parquet
- Avro
- Protobuf
- Log files

It uses open-source Python libraries (`pandas`, `pyarrow`/`fastparquet`, `fastavro`, `protobuf`) to provide a unified view of the data.


## Read JSONL

In [ ]:
import json
import pandas as pd
from pathlib import Path

jsonl_path = Path("data/samples/hot/temperature") / next(Path("data/samples/hot/temperature").rglob("*.jsonl")).name

print("Reading JSONL from:", jsonl_path)

records = []
with open(jsonl_path) as f:
    for line in f:
        if line.strip():
            records.append(json.loads(line))

df_jsonl = pd.DataFrame(records)
df_jsonl.head()


## Read Parquet

In [ ]:
import pandas as pd

parquet_path = list(Path("data/samples/batch").rglob("*.parquet"))[0]
print("Reading Parquet from:", parquet_path)

try:
    df_parquet = pd.read_parquet(parquet_path)
    display(df_parquet.head())
except Exception as e:
    print("Parquet read failed (engine missing):", e)


## Validate against Avro Schema

In [ ]:
import fastavro

avro_schema_path = Path("schema/temperature.avsc")
schema = None
with open(avro_schema_path) as f:
    import json
    schema = json.load(f)

print("Loaded Avro schema:", avro_schema_path)

# Example: validate records
for rec in df_jsonl.to_dict(orient="records"):
    fastavro.validation.validate(rec, schema)
print("Validation passed for JSONL records against Avro schema")


## Inspect Protobuf Schema

In [ ]:

# Load .proto definition manually (simple)
proto_file = Path("proto/temperature.proto")
print("Proto file at:", proto_file)

print(proto_file.read_text())


## Read Ops Log

In [ ]:
ops_log_path = list(Path("data/samples/hot/ops").rglob("*.log"))[0]
print("Reading log file:", ops_log_path)
with open(ops_log_path) as f:
    for line in f.readlines():
        print(line.strip())
